In [ ]:
# Binary classification on circular data using a simple neural network

# Setting GPU: Notebook settings -> T3 GPU

import torch
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
from sklearn import datasets

# Generate synthetic circular data
n_pts = 500
X, y = datasets.make_circles(n_samples=n_pts, random_state=123, noise=0.1, factor=0.2)

x_data = torch.Tensor(X)
y_data = torch.Tensor(y.reshape(500, 1))

print(x_data.shape, y_data.shape)  # batch size x input dimension

# Plot data points by class
def scatter_plot():
    plt.scatter(X[y == 0, 0], X[y == 0, 1], color='red')
    plt.scatter(X[y == 1, 0], X[y == 1, 1], color='blue')

scatter_plot()

# Define a simple neural network model
class Model(nn.Module):
    def __init__(self, input_size, H1, output_size):
        super().__init__()
        self.linear = nn.Linear(input_size, H1)
        self.linear2 = nn.Linear(H1, output_size)

    def forward(self, x):
        x = torch.sigmoid(self.linear(x))
        x = torch.sigmoid(self.linear2(x))
        return x

    def predict(self, x):
        return 1 if self.forward(x) >= 0.5 else 0

model = Model(2, 4, 1)
print(list(model.parameters()))

# Loss and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.1)

# Training loop
epochs = 1000
losses = []

for i in range(epochs):
    optimizer.zero_grad()
    y_pred = model.forward(x_data)
    y_pred = model(x_data)
    loss = criterion(y_pred, y_data)

    print("epoch:", i, "loss:", loss.item())

    losses.append(loss.item())
    loss.backward()
    optimizer.step()

# Plot training loss
plt.plot(range(epochs), losses)
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.title('Training Loss')

# Decision boundary visualization
def plot_decision_boundary(X, y):
    x_span = np.linspace(min(X[:, 0]), max(X[:, 0]))
    y_span = np.linspace(min(X[:, 1]), max(X[:, 1]))
    xx, yy = np.meshgrid(x_span, y_span)
    grid = torch.Tensor(np.c_[xx.ravel(), yy.ravel()])
    pred_func = model.forward(grid)
    z = pred_func.view(xx.shape).detach().numpy()
    plt.contourf(xx, yy, z)

plot_decision_boundary(x_data, y_data)
scatter_plot()